In [1]:
import os 
from collections import defaultdict

from tqdm import tqdm
from sklearn.model_selection import train_test_split

from utils.train import train_set
from data.read_data import read_data_file


ModuleNotFoundError: No module named 'tqdm'

In [2]:
yaml_path = "./config/train.yaml"
CONFIG,Logger,DEVICE = train_set(yaml_path,experimentName = None,upload = False,filename = "./test/logs",is_notebook = True)

CONFIG["run_db"] = False
CONFIG["num_hidden_layers"] = 12
CONFIG["DATA_NUM_RATE"] = 0.2
CONFIG["EPOCHS"] = 35
CONFIG["DEBUG_MODEL"] = False

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: adreambear. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
binary_word_list = []
with open("/root/autodl-tmp/epsave/data/binary/classifyword.txt","r") as fin:
    for line in fin:
        binary_word_list.append(line.strip())

In [5]:
input_dir = CONFIG["INPUT_DIR"]
positive_file = os.path.join(input_dir,"PositiveData")
head = ["Query","keyword","query_len","emotion_value","label"]
positive_df = read_data_file(positive_file,False,5000,head = head)

In [6]:
positive_df.shape

(10569, 5)

In [7]:
input_dir = CONFIG["INPUT_DIR"]
negative_file = os.path.join(input_dir,"NegativeData")
head = ["Query","keyword","query_len","emotion_value","label"]
# negative_df = read_data_file(negative_file,False,5000,head = head)


In [8]:
# negative_df.shape


In [10]:
with open(os.path.join(CONFIG["INPUT_DIR"],"Schema"),"r") as f:
       line=f.readlines()[0].strip()
       column_name=line.split("\t")
       CONFIG["Logger"].info(f"column_name :{column_name}")
       CONFIG["COLUMN_NAME"]=column_name

column_name :['Query', 'keyword', 'query_len', 'emotion_value', 'label']


In [11]:
# write_file = os.path.join(CONFIG["INPUT_DIR"],"PositiveDataTemp")
# with open(write_file,'w') as write_object:
#     for id, rows in positive_df.iterrows():
#         write_object.write('\t'.join(rows) + "\n")
# write_file = os.path.join(CONFIG["INPUT_DIR"],"NegativeDataTemp")
# with open(write_file,'w') as write_object:
#     for id, rows in negative_df.iterrows():
#         write_object.write('\t'.join(rows) + "\n")

In [12]:
keyword_file = "/root/autodl-tmp/epsave/data/raw_data/euphemism_answer_drug.txt"
euphemism_answer = defaultdict(list)
with open(keyword_file, 'r') as fin:
    for line in fin:
        ans = line.split(':')[0].strip().lower()
        for i in line.split(':')[1].split(';'):
            euphemism_answer[i.strip().lower()].append(ans)

In [13]:
binary_word_list

['candy',
 'beans',
 'go',
 'superman',
 'coffee',
 'phoenix',
 'tabs',
 'live',
 'bars',
 'diesel',
 'truck drivers',
 'food',
 'fire',
 'girl',
 'white',
 'wet',
 'zip',
 'junk',
 'leaf',
 'shoes',
 'flour',
 'snowflake',
 'green',
 'space',
 'trees',
 'paint',
 'lemonade',
 'work',
 'k',
 'purple',
 'poison',
 'black',
 'blue',
 'salt',
 'jet',
 'ace',
 'buttons',
 'whites',
 'him',
 'bananas',
 'blues',
 'rope',
 'mother',
 'scratch',
 'mph',
 'water',
 'speed']

In [14]:
head = CONFIG['COLUMN_NAME']


In [15]:
keyword_index = head.index('keyword')
query_index = head.index('Query')
query_len_index = head.index('query_len')

In [16]:
# 处理正样本

for binary_word in tqdm(binary_word_list):
    keyword_list = euphemism_answer[binary_word]
    train_output_list = []
    test_output_list = []
    val_output_list = []

    for id, rows in positive_df.iterrows():
        if any( keyword == rows[keyword_index] for keyword in keyword_list):
            output_rows = rows.copy()
            output_rows[keyword_index] = binary_word
            output_rows[query_index] = output_rows[query_index].replace('[MASK]',binary_word)
            if int(output_rows[query_len_index]) %5 == 0:
                test_output_list.append(output_rows)
            elif int(output_rows[query_len_index]) %4 == 0:
                val_output_list.append(output_rows)
            else:
                train_output_list.append(output_rows)

    write_path = os.path.join(CONFIG["INPUT_DIR"],binary_word)
    if not os.path.exists(write_path):
        os.mkdir(write_path)
    
    with open(os.path.join(write_path,"PositiveTrain"),'w') as write_object:
        for write_rows in train_output_list:
            write_object.write('\t'.join(write_rows) + "\n")

    with open(os.path.join(write_path,"PositiveTest"),'w') as write_object:
        for write_rows in test_output_list:
            write_object.write('\t'.join(write_rows) + "\n")

    with open(os.path.join(write_path,"PositiveVal"),'w') as write_object:
        for write_rows in val_output_list:
            write_object.write('\t'.join(write_rows) + "\n")

    print(f'keyword: {binary_word} , train_num: {len(train_output_list)} test_num: {len(test_output_list)} \
        rate:  {len(test_output_list)/len(train_output_list)}' ) 

  2%|▏         | 1/47 [00:00<00:44,  1.03it/s]

keyword: candy , train_num: 531 test_num: 214         rate:  0.4030131826741996


  4%|▍         | 2/47 [00:01<00:37,  1.20it/s]

keyword: beans , train_num: 253 test_num: 110         rate:  0.43478260869565216


  6%|▋         | 3/47 [00:02<00:31,  1.38it/s]

...

keyword: mph , train_num: 20 test_num: 1         rate:  0.05


 98%|█████████▊| 46/47 [00:34<00:00,  1.48it/s]

keyword: water , train_num: 121 test_num: 40         rate:  0.3305785123966942


100%|██████████| 47/47 [00:35<00:00,  1.34it/s]

keyword: speed , train_num: 99 test_num: 25         rate:  0.25252525252525254


In [15]:
negative_output_dict = defaultdict(list)

for index, rows in negative_df.iterrows():
    if rows[keyword_index] in binary_word_list:
        binary_word = rows[keyword_index]
        output_rows = rows.copy()
        output_rows[query_index] = output_rows[query_index].replace('[MASK]',binary_word)
        
        if index %2 == 0:
            key_name = 'unlabel'+binary_word
        else:
            if int(output_rows[query_len_index]) %5 == 0:
                key_name = 'test'+binary_word
            elif int(output_rows[query_len_index]) %4 == 0:
                key_name = 'val'+binary_word
            else:
                key_name = 'train'+binary_word
        negative_output_dict[key_name].append(output_rows)

KeyboardInterrupt: 

In [ ]:
train_num_sum = 0
for binary_word in tqdm(binary_word_list):
    write_path = os.path.join(CONFIG["INPUT_DIR"],binary_word)
    if not os.path.exists(write_path):
        os.mkdir(write_path)
    
    with open(os.path.join(write_path,"NegativeTrain"),'w') as write_object:
        key_name = 'train'+binary_word
        train_num = len(negative_output_dict[key_name])
        for write_rows in negative_output_dict[key_name]:
            write_object.write('\t'.join(write_rows) + "\n")

    with open(os.path.join(write_path,"NegativeTest"),'w') as write_object:
        key_name = 'test'+binary_word
        test_num = len(negative_output_dict[key_name])

        for write_rows in negative_output_dict[key_name]:
            write_object.write('\t'.join(write_rows) + "\n")
    with open(os.path.join(write_path,"NegativeVal"),'w') as write_object:
        key_name = 'val'+binary_word
        test_num = len(negative_output_dict[key_name])

        for write_rows in negative_output_dict[key_name]:
            write_object.write('\t'.join(write_rows) + "\n")
    
    with open(os.path.join(write_path,"UnlabelData"),'w') as write_object:
        key_name = 'unlabel'+binary_word
        unlabel_num = len(negative_output_dict[key_name])

        for write_rows in negative_output_dict[key_name]:
            write_object.write('\t'.join(write_rows) + "\n")
    train_num_sum += train_num
    
    print(f'keyword: {binary_word} , train_num: {train_num} test_num: {test_num} unlabel_num: {unlabel_num} \
        test rate:  {test_num/train_num} unlabel rate:  {unlabel_num/(train_num+test_num)}' ) 

  0%|          | 0/47 [00:00<?, ?it/s]

keyword: candy , train_num: 125 test_num: 39 unlabel_num: 205         test rate:  0.312 unlabel rate:  1.25
keyword: beans , train_num: 64 test_num: 25 unlabel_num: 115         test rate:  0.390625 unlabel rate:  1.2921348314606742


 28%|██▊       | 13/47 [00:00<00:00, 36.69it/s]

keyword: go , train_num: 18507 test_num: 5687 unlabel_num: 30281         test rate:  0.3072891338412493 unlabel rate:  1.251591303628999
keyword: superman , train_num: 14 test_num: 8 unlabel_num: 26         test rate:  0.5714285714285714 unlabel rate:  1.1818181818181819
keyword: coffee , train_num: 1202 test_num: 468 unlabel_num: 2076         test rate:  0.389351081530782 unlabel rate:  1.2431137724550898
keyword: phoenix , train_num: 19 test_num: 7 unlabel_num: 30         test rate:  0.3684210526315789 unlabel rate:  1.1538461538461537
keyword: tabs , train_num: 24 test_num: 8 unlabel_num: 40         test rate:  0.3333333333333333 unlabel rate:  1.25
keyword: live , train_num: 2316 test_num: 758 unlabel_num: 3920         test rate:  0.32728842832469773 unlabel rate:  1.2752114508783343
keyword: bars , train_num: 68 test_num: 18 unlabel_num: 114         test rate:  0.2647058823529412 unlabel rate:  1.3255813953488371
keyword: diesel , train_num: 5 test_num: 4 unlabel_num: 12         t

100%|██████████| 47/47 [00:00<00:00, 50.86it/s]

keyword: work , train_num: 12934 test_num: 4212 unlabel_num: 21858         test rate:  0.3256533168393382 unlabel rate:  1.274816283681325
keyword: k , train_num: 148 test_num: 43 unlabel_num: 224         test rate:  0.2905405405405405 unlabel rate:  1.1727748691099475
keyword: purple , train_num: 148 test_num: 49 unlabel_num: 245         test rate:  0.3310810810810811 unlabel rate:  1.2436548223350254
keyword: poison , train_num: 49 test_num: 8 unlabel_num: 64         test rate:  0.16326530612244897 unlabel rate:  1.1228070175438596
keyword: black , train_num: 595 test_num: 188 unlabel_num: 955         test rate:  0.31596638655462184 unlabel rate:  1.2196679438058748
keyword: blue , train_num: 371 test_num: 155 unlabel_num: 663         test rate:  0.41778975741239893 unlabel rate:  1.2604562737642586
keyword: salt , train_num: 66 test_num: 15 unlabel_num: 104         test rate:  0.22727272727272727 unlabel rate:  1.2839506172839505
keyword: jet , train_num: 89 test_num: 32 unlabel_num

In [ ]:
# 产生swap id
# from data.read_data import select_data,select_data,TrainData,file_split,DataByWord
# positive_nums = []
# negative_nums = []
# all_nums = []
# trainDataObj=DataByWord(CONFIG,"train",column_name=column_name)
# for i in range(trainDataObj.iter_num):
#     df = trainDataObj.get_data()
#     positive_num = df.query("label == 1").shape[0]
#     negative_num = df.query("label == 0").shape[0]
#     positive_nums.append(positive_num)
#     negative_nums.append(negative_num)
#     all_nums.append(positive_num + negative_num)
# swap_id = trainDataObj.iter_num *[0]
# diff_num = [positive_nums[i]-negative_nums[i] for i in range(trainDataObj.iter_num)]
# save_swap_id = trainDataObj.iter_num *[0]
# sorted_diff_num = sorted(diff_num)

# list1 = []
# sum1 = 0
# list2 = []
# sum2 = 0
# n = len(diff_num)
# for i in range(n):
#     if sum1 >= sum2 and diff_num[i] > 0:
        
#         list2.append(diff_num[i])
#         sum2 += diff_num[i]
#     elif sum1 >= sum2 and diff_num[i] <= 0:
#         list1.append(diff_num[i])
#         sum1 += diff_num[i]
#     elif sum1 <= sum2 and diff_num[i] > 0:
#         list1.append(diff_num[i])
#         sum1 += diff_num[i]
#     else:
#         list2.append(diff_num[i])
#         sum2 += diff_num[i]
# swap_id = [ 1 if diff_num[i] in list1 else 0 for i in range(n)]
# with open(os.path.join(os.path.dirname(CONFIG["INPUT_DIR"]),"swap_id"),"w") as write_object:
#     for id in swap_id:
#         write_object.write(str(id)+"\n")


In [17]:
# 正样本变换
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [18]:
word_num = len(binary_word_list)
aug = naw.SynonymAug(aug_src='wordnet')
special_token = "$"

In [19]:
for i,word_name in enumerate(tqdm(binary_word_list)):
    for write_type in ["train","val","test"]:
        positive_word_file = os.path.join(CONFIG["INPUT_DIR"],word_name,"Positive"+write_type.capitalize())
        positive_word_df = read_data_file(positive_word_file,False,5000,head = head) 
        query = positive_word_df.apply(lambda row :row[query_index].replace(row[keyword_index],special_token),axis = 1).values.tolist()

        aug_query = []
        for text in query:
            aug_query.append(aug.augment(text,n = word_num)[i])

        aug_query = [ text.replace(special_token,word_name) for text in aug_query]
        positive_word_df["Query"] = aug_query
        output_list = positive_word_df.values.tolist()

        write_word_file = os.path.join(CONFIG["INPUT_DIR"],word_name,"Positive"+write_type.capitalize()+"Aug")
        with open(write_word_file,'w') as write_object:
            for write_rows in output_list:
                write_object.write('\t'.join(write_rows) + "\n")


 30%|██▉       | 14/47 [39:07<2:24:21, 262.46s/it]

100%|██████████| 1070/1070 [01:17<00:00, 13.78it/s]
